In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Параметры задачи
DOMAIN_LENGTH = 8.0
FINAL_TIME = 3.0
WAVE_SPEED = 0.5

# Начальное условие
def start_condition(x):
    return np.sin(4 * np.pi * x / DOMAIN_LENGTH)

# Аналитическое решение
def true_solution(x, t):
    return start_condition((x - WAVE_SPEED * t) % DOMAIN_LENGTH)

# Схема уголок
def upwind_method(u, speed, dt, dx):
    u_updated = u.copy()
    factor = speed * dt / dx
    u_updated[1:] = u[1:] - factor * (u[1:] - u[:-1])
    u_updated[0] = u[0] - factor * (u[0] - u[-1])
    return u_updated

# Схема прямоугольник
def central_diff_method(u, speed, dt, dx):
    u_updated = u.copy()
    factor = speed * dt / (2 * dx)
    u_updated[1:-1] = u[1:-1] - factor * (u[2:] - u[:-2])
    u_updated[0] = u[0] - factor * (u[1] - u[-1])
    u_updated[-1] = u[-1] - factor * (u[0] - u[-2])
    return u_updated

# Схема Лакс-Вендрофф
def lax_wendroff_method(u, speed, dt, dx):
    u_updated = u.copy()
    sigma = speed * dt / dx
    sigma_sq = sigma**2
    u_updated[1:-1] = u[1:-1] - (sigma / 2) * (u[2:] - u[:-2]) + (sigma_sq / 2) * (u[2:] - 2 * u[1:-1] + u[:-2])
    u_updated[0] = u[0] - (sigma / 2) * (u[1] - u[-1]) + (sigma_sq / 2) * (u[1] - 2 * u[0] + u[-1])
    u_updated[-1] = u[-1] - (sigma / 2) * (u[0] - u[-2]) + (sigma_sq / 2) * (u[0] - 2 * u[-1] + u[-2])
    return u_updated

# Численное решение
NUM_NODES = 120
dx = DOMAIN_LENGTH / (NUM_NODES - 1)
COURANT_FACTOR = 0.8
dt = COURANT_FACTOR * dx / WAVE_SPEED
NUM_TIME_STEPS = int(FINAL_TIME / dt)

x_values = np.linspace(0, DOMAIN_LENGTH, NUM_NODES)
u_true = start_condition(x_values)
u_upwind = u_true.copy()
u_central = u_true.copy()
u_lw = u_true.copy()

# Хранение истории решений
upwind_solutions = [u_upwind.copy()]
central_solutions = [u_central.copy()]
lw_solutions = [u_lw.copy()]
true_solutions = [u_true.copy()]

# Интегрирование во времени
for step in range(NUM_TIME_STEPS):
    time = (step + 1) * dt
    u_true = true_solution(x_values, time)
    u_upwind = upwind_method(u_upwind, WAVE_SPEED, dt, dx)
    u_central = central_diff_method(u_central, WAVE_SPEED, dt, dx)
    u_lw = lax_wendroff_method(u_lw, WAVE_SPEED, dt, dx)
    upwind_solutions.append(u_upwind.copy())
    central_solutions.append(u_central.copy())
    lw_solutions.append(u_lw.copy())
    true_solutions.append(u_true.copy())

# График сравнения решений
plt.figure(figsize=(9, 5), facecolor='lightgray')
plt.title("Решения уравнения переноса", fontsize=16, family='serif', pad=20)
plt.xlabel("Координата", fontsize=14, family='serif')
plt.ylabel("Амплитуда", fontsize=14, family='serif')
plt.plot(x_values, true_solutions[-1], color='#2E86C1', linestyle='-', marker='o', markersize=4, label='Аналитическое', alpha=0.9)
plt.plot(x_values, upwind_solutions[-1], color='#F39C12', linestyle='-', marker='s', markersize=4, label='Уголок', alpha=0.9)
plt.plot(x_values, central_solutions[-1], color='#E74C3C', linestyle='-', marker='^', markersize=4, label='Центральная', alpha=0.9)
plt.plot(x_values, lw_solutions[-1], color='#27AE60', linestyle='-', marker='d', markersize=4, label='Лакс-Вендрофф', alpha=0.9)
plt.legend(loc='upper right', fontsize=12, frameon=True, edgecolor='black', facecolor='white', framealpha=0.95)
plt.grid(True, linestyle=':', color='gray', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Анализ сходимости
node_counts = [40, 80, 160, 320, 640, 1280]
upwind_errors = []
central_errors = []
lw_errors = []

for nodes in node_counts:
    dx = DOMAIN_LENGTH / (nodes - 1)
    dt = COURANT_FACTOR * dx / WAVE_SPEED
    num_steps = int(FINAL_TIME / dt)
    x_values = np.linspace(0, DOMAIN_LENGTH, nodes)
    u_true = true_solution(x_values, FINAL_TIME)
    # Уголок
    u_upwind = start_condition(x_values)
    for step in range(num_steps):
        u_upwind = upwind_method(u_upwind, WAVE_SPEED, dt, dx)
    error = np.sqrt(np.sum((u_upwind - u_true)**2) * dx)
    upwind_errors.append(error)
    # Центральная разность
    u_central = start_condition(x_values)
    for step in range(num_steps):
        u_central = central_diff_method(u_central, WAVE_SPEED, dt, dx)
    error = np.sqrt(np.sum((u_central - u_true)**2) * dx)
    central_errors.append(error)
    # Лакс-Вендрофф
    u_lw = start_condition(x_values)
    for step in range(num_steps):
        u_lw = lax_wendroff_method(u_lw, WAVE_SPEED, dt, dx)
    error = np.sqrt(np.sum((u_lw - u_true)**2) * dx)
    lw_errors.append(error)

# График сходимости и порядка аппроксимации
dx_values = [DOMAIN_LENGTH / (nodes - 1) for nodes in node_counts]
plt.figure(figsize=(7, 4), facecolor='lightgray')
plt.title("Сходимость численных методов", fontsize=16, family='serif', pad=20)
plt.xlabel("Размер шага dx", fontsize=14, family='serif')
plt.ylabel("Ошибка (L2)", fontsize=14, family='serif')
plt.loglog(dx_values, upwind_errors, color='#F4D03F', linestyle='-', marker='x', markersize=8, label='Уголок', linewidth=2)
plt.loglog(dx_values, central_errors, color='#C39BD3', linestyle='-', marker='+', markersize=8, label='Центральная', linewidth=2)
plt.loglog(dx_values, lw_errors, color='#82E0AA', linestyle='-', marker='*', markersize=8, label='Лакс-Вендрофф', linewidth=2)
dx_theory = np.array(dx_values)
plt.loglog(dx_theory, 3*dx_theory, color='darkgray', linestyle='-.', label='O(dx)', linewidth=1.5)
plt.loglog(dx_theory, 3*dx_theory**2, color='black', linestyle='--', label='O(dx²)', linewidth=1.5)
plt.legend(loc='lower right', fontsize=12, frameon=True, edgecolor='black', facecolor='white', framealpha=0.95)
plt.grid(True, linestyle=':', color='gray', alpha=0.6)
plt.tight_layout()
plt.show()

# Вывод ошибок
print("L2-ошибки для методов:")
for i, nodes in enumerate(node_counts):
    print(f"Число узлов = {nodes}:")
    print(f"  Уголок: {upwind_errors[i]:.6e}")
    print(f"  Центральная: {central_errors[i]:.6e}")
    print(f"  Лакс-Вендрофф: {lw_errors[i]:.6e}")